<a href="https://colab.research.google.com/github/adamrhurley/FYP/blob/main/MusicRecommenderSystem/svd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://blog.cambridgespark.com/tutorial-practical-introduction-to-recommender-systems-dbe22848392b
# https://predictivehacks.com/how-to-run-recommender-systems-in-python/

In [ ]:
pip install surprise

     |████████████████████████████████| 11.8MB 9.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617595 sha256=400a3901a473489fee333d1656f56edfea5c0e8553a389fc84eba01900c5895a
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving artists.csv to artists.csv
Saving userArtists.csv to userArtists.csv


In [ ]:
import io
users = pd.read_csv(io.BytesIO(uploaded['userArtists.csv']))
artists = pd.read_csv(io.BytesIO(uploaded['artists.csv']))

In [ ]:
mergedDF = pd.merge(users, artists, on='artistID')
mergedDF.head()

,userID,artistID,weight,name,url,pictureURL
0,2,51,13883,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg
1,4,51,228,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg
2,27,51,85,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg
3,28,51,10,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg
4,62,51,528,Duran Duran,http://www.last.fm/music/Duran+Duran,http://userserve-ak.last.fm/serve/252/155668.jpg


In [ ]:
users.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [ ]:
artists.head()

,artistID,name,url,pictureURL
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


In [ ]:
reader = Reader(rating_scale = (0,100000))

data = Dataset.load_from_df(users[['userID', 'artistID', 'weight']], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    99347.547799344.734099305.569499314.181999322.687499344.012399312.666399334.049199312.718299286.186999322.435318.9334 
MAE (testset)     99297.294499299.770799273.491399284.786499280.313399295.436199275.417499304.564599288.908099243.926499284.390816.7508 
Fit time          4.38    4.31    4.32    4.29    4.27    4.32    4.33    4.35    4.27    4.32    4.32    0.03    
Test time         0.07    0.07    0.16    0.07    0.07    0.07    0.07    0.16    0.07    0.08    0.09    0.03    


{'fit_time': (4.379277944564819,
  4.3099095821380615,
  4.3178551197052,
  4.29436731338501,
  4.266406536102295,
  4.3186562061309814,
  4.329766511917114,
  4.348835706710815,
  4.273674964904785,
  4.321028232574463),
 'test_mae': array([99297.29435125, 99299.77068074, 99273.49127531, 99284.78640672,
        99280.31327079, 99295.43606593, 99275.41741967, 99304.56447269,
        99288.90800388, 99243.92642465]),
 'test_rmse': array([99347.54766843, 99344.73400922, 99305.56944778, 99314.18189152,
        99322.68742127, 99344.01229756, 99312.66633756, 99334.04914753,
        99312.71824252, 99286.18692475]),
 'test_time': (0.07043957710266113,
  0.07245230674743652,
  0.15595579147338867,
  0.06908988952636719,
  0.07328200340270996,
  0.06797099113464355,
  0.06873488426208496,
  0.15668869018554688,
  0.0741724967956543,
  0.0783541202545166)}

In [ ]:
userDF = users[(users['userID'] == 35)]
userDF = userDF.set_index('artistID')
userDF = userDF.join(artists)['name']
userDF.head(userDF.shape[0])

artistID
186                     Elton John
212                        Man Man
444              She Wants Revenge
920                          Viper
964                          Hevia
1169                  Lunatic Soul
1170                  Billy Talent
1171               Escape The Fate
1172                   Silverstein
1173    The Red Jumpsuit Apparatus
1174                 You Me At Six
1175                  The Blackout
1176                       Alesana
1177                        Sum 41
1178             Dance Gavin Dance
Name: name, dtype: object

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
artists = artists.copy()

artists['EstimatePlays'] = artists['artistID'].apply(lambda x: svd.predict(35, x).est)
artists = artists.sort_values(by=['EstimatePlays'], ascending=False)

In [ ]:
artists.head(10)

,artistID,name,url,pictureURL,EstimatePlays
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg,100000
11752,12178,BEAT!BEAT!BEAT!,http://www.last.fm/music/BEAT%21BEAT%21BEAT%21,http://userserve-ak.last.fm/serve/252/51885229...,100000
11758,12184,ãƒˆã‚¯ãƒžãƒ«ã‚·ãƒ¥ãƒ¼ã‚´,http://www.last.fm/music/%E3%83%88%E3%82%AF%E3...,http://userserve-ak.last.fm/serve/252/12556183...,100000
11757,12183,Thao with The Get Down Stay Down,http://www.last.fm/music/Thao+with+The+Get+Dow...,http://userserve-ak.last.fm/serve/252/43814189...,100000
11756,12182,ãƒªã‚¢ãƒ»ãƒ‡ã‚£ã‚¾ãƒ³,http://www.last.fm/music/%E3%83%AA%E3%82%A2%E3...,http://userserve-ak.last.fm/serve/252/19825455...,100000
11755,12181,Maria Cecilia e Rodolfo - Mato e morro por vocÃª,http://www.last.fm/music/Maria+Cecilia+e+Rodol...,http://userserve-ak.last.fm/serve/252/34953415...,100000
11754,12180,The Weird Sisters,http://www.last.fm/music/The+Weird+Sisters,http://userserve-ak.last.fm/serve/252/4637618.jpg,100000
11753,12179,maria cesilha e rodolfo,http://www.last.fm/music/maria+cesilha+e+rodolfo,NaN,100000
11751,12177,Guilherma e santiago,http://www.last.fm/music/Guilherma+e+santiago,NaN,100000
11777,12203,Fat Freddy's Drop,http://www.last.fm/music/Fat+Freddy%27s+Drop,http://userserve-ak.last.fm/serve/252/5273598.jpg,100000
